### Select your source mapping file
Mapping file must be excel file that conforms to the mapping standard

In [ ]:
import pandas as pd
import fedelemflowlist
from fedelemflowlist.globals import flowmappingpath, flowmapping_fields
import os
import ipywidgets as widgets
from IPython.display import display, FileLink

if not os.path.exists('./scratch'):
    os.mkdir('./scratch')


In [ ]:
uploader = widgets.FileUpload(
     accept   = '.csv'
    ,multiple = False
)

display(uploader)


In [ ]:
if not uploader.value:
    raise ValueError('Input Excel file is required to continue.')

input_file = next(iter(uploader.value))
input_full = r"./scratch/" + input_file

if os.path.exists(input_full):
    os.remove(input_full)
    
with open(input_full,"wb") as fp:
    fp.write(uploader.value[input_file]['content']);
    

In [ ]:
mapping = pd.read_csv(input_full)
mapping_length = len(mapping)
print(mapping_length)
all_flows = fedelemflowlist.get_flows()
all_flows = all_flows[['Flowable', 'Context', 'Flow UUID', 'Unit']]
mapping_w_flowinfo = pd.merge(mapping, all_flows,
            left_on=['TargetFlowName', 'TargetFlowContext', 'TargetUnit'],
            right_on=['Flowable', 'Context', 'Unit'])
mapping_w_flowinfo = mapping_w_flowinfo.drop(columns=['TargetFlowUUID', 'Flowable',
                                        'Context', 'Unit'])
mapping_w_flowinfo = mapping_w_flowinfo.rename(columns={'Flow UUID': 'TargetFlowUUID'})
mapping_merged_len = len(mapping_w_flowinfo)
if mapping_length > mapping_merged_len:
    print("Not all flows were mapped to flows in the list")

flowmapping_order = list(flowmapping_fields.keys())
mapping_w_flowinfo = mapping_w_flowinfo[flowmapping_order]
print(mapping_w_flowinfo)

In [ ]:
output_file = 'flows_w_mappings.zip'
output_full = r"scratch/" + output_file

if os.path.exists(output_full):
   os.remove(output_full)

In [ ]:
mapping_flow_uuids = pd.DataFrame(pd.unique(mapping_w_flowinfo['TargetFlowUUID']),columns=["Flow UUID"])

#Get all flows
all_flows = fedelemflowlist.get_flows()
#Subset all flows to get just those used in selected mapping
flows_used_in_mapping =  pd.merge(all_flows,mapping_flow_uuids)

fedelemflowlist.write_jsonld(flows_used_in_mapping,output_full,mapping_w_flowinfo)
print("mapping file created as json")
download = FileLink(output_full,result_html_prefix="Click here to download: ")
display(download)
